In [32]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
import numpy as np

In [18]:
train_data_dir = 'dataset'

In [19]:
img_height, img_width = 400, 400  # Use the pixel size of your images
batch_size = 32
num_classes = 3

In [20]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2 
)

In [21]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',  # For multi-class classification
    subset='training'
)

Found 276 images belonging to 3 classes.


In [22]:
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 68 images belonging to 3 classes.


In [23]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Output layer with softmax for probabilities
])

In [24]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
epochs = 10  # You can adjust the number of epochs
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/10
17/17 [==============================] - 49s 3s/step - loss: 4.2592 - accuracy: 0.6192 - val_loss: 0.4544 - val_accuracy: 0.7969
Epoch 2/10
17/17 [==============================] - 48s 3s/step - loss: 0.2833 - accuracy: 0.8692 - val_loss: 0.4178 - val_accuracy: 0.7969
Epoch 3/10
17/17 [==============================] - 49s 3s/step - loss: 0.1830 - accuracy: 0.9346 - val_loss: 0.2170 - val_accuracy: 0.9375
Epoch 4/10
17/17 [==============================] - 53s 3s/step - loss: 0.0892 - accuracy: 0.9731 - val_loss: 0.2544 - val_accuracy: 0.9531
Epoch 5/10
17/17 [==============================] - 51s 3s/step - loss: 0.0586 - accuracy: 0.9769 - val_loss: 0.5357 - val_accuracy: 0.8594
Epoch 6/10
17/17 [==============================] - 54s 3s/step - loss: 0.0326 - accuracy: 0.9963 - val_loss: 0.4444 - val_accuracy: 0.8125
Epoch 7/10
17/17 [==============================] - 51s 3s/step - loss: 0.0232 - accuracy: 0.9926 - val_loss: 0.1393 - val_accuracy: 0.9688
Epoch 8/10
17/17 [==

In [26]:
test_loss, test_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {test_accuracy}")

5/5 [==============================] - 3s 482ms/step - loss: 0.4548 - accuracy: 0.8824
Validation Accuracy: 0.8823529481887817


In [28]:
model.save('character_recognition_model.h5')

In [39]:
def predict_image(model_path, image_path, target_size):
    loaded_model = tf.keras.models.load_model(model_path)
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize the image

    predictions = loaded_model.predict(img_array)
    predicted_class_index = np.argmax(predictions)
     # Get the mapping from class indices to labels from the training generator
    class_labels = {v: k for k, v in train_generator.class_indices.items()}
    print(class_labels)
    predicted_class_label = class_labels[predicted_class_index]

    return predicted_class_label


In [44]:
model_path = 'character_recognition_model.h5'  # Path to your saved model
single_image_path = 'test_images\image.png'  # Path to the image you want to predict

In [45]:
predicted_class = predict_image(model_path, single_image_path, (img_height, img_width))
print(f"\nPredicted class for '{single_image_path}': {predicted_class}")

1/1 [==============================] - 0s 167ms/step
{0: 'Minun_samePixel_data', 1: 'Plusle_samePixel_data', 2: 'pikachu_samePixel_data'}

Predicted class for 'test_images\image.png': pikachu_samePixel_data
